<a href="https://colab.research.google.com/github/purnaa-l/MiniProject-ML-Models-All/blob/main/Biobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers flask torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import requests
# Load BioBERT fine-tuned for QA (from dmis-lab)
model_name = "dmis-lab/biobert-base-cased-v1.1-squad"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


def search_wikipedia(query):
    url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&exintro&explaintext&titles={query}"
    response = requests.get(url)
    data = response.json()

    # Extracting the page extract (first few lines of the article)
    pages = data['query']['pages']
    for page_id in pages:
        extract = pages[page_id].get('extract', 'No content available.')
        return extract

# Example input
query=torch.are_deterministic_algorithms_enabled
context = search_wikipedia(query)

# # Encode inputs
# inputs = tokenizer.encode_plus(question, context, return_tensors="pt")

# # Get start and end scores
# with torch.no_grad():
#     outputs = model(**inputs)
#     start_scores = outputs.start_logits
#     end_scores = outputs.end_logits

# # Decode answer
# start_idx = torch.argmax(start_scores)
# end_idx = torch.argmax(end_scores) + 1  # add 1 to include the last token

# answer = tokenizer.convert_tokens_to_string(
#     tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx])
# )

# print("Question:", question)
print("Answer:", context)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Answer: No content available.


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
import requests

# Load BioBERT fine-tuned for QA (SQuAD format)
model_name = "dmis-lab/biobert-base-cased-v1.1-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


# Function to get context from Wikipedia
def search_wikipedia(query):
    url = f"https://en.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&exintro&explaintext&titles={query}"
    response = requests.get(url)
    data = response.json()

    pages = data['query']['pages']
    for page_id in pages:
        extract = pages[page_id].get('extract', 'No content available.')
        return extract

# Set your question
question = "How is asthma treated?"

# Use Wikipedia to fetch relevant context
context = search_wikipedia("Asthma")

# Encode the inputs for the model
inputs = tokenizer.encode_plus(
    question,
    context,
    return_tensors="pt",
    truncation=True,
    max_length=512,
    padding="max_length"
)

# Get start and end logits
with torch.no_grad():
    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

# Get most likely start and end positions
start_idx = torch.argmax(start_scores)
end_idx = torch.argmax(end_scores) + 1

# Decode the predicted tokens into the final answer
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx])
)

print(f"Question: {question}")
print(f"Answer: {answer}")


Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Question: How is asthma treated?
Answer: treatment of rapidly worsening symptoms is usually with an inhaled short - acting beta2 agonist such as salbutamol and corticosteroids taken by mouth


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Question: How is asthma treated?
Answer: is asthma treated? [SEP]
